In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import style
from scipy import fftpack
import seaborn as sns
style.use('seaborn')

In [2]:
def rename_column(df,prefix):
    column_name = list(df.columns)
    column_name = [prefix + name for name in column_name]
    return df.set_axis(column_name, axis=1)

In [3]:
target_set = pd.read_csv(".\\bearing_classes.csv",sep=";")
origin_set = pd.read_csv(".\\bearing_signals.csv")

In [4]:
df_train = origin_set.drop(['experiment_id','bearing_1_id','rpm','hz','w', 'timestamp'],axis=1)

In [5]:
df_bearing_1_x = df_train.drop(['a2_x','a2_y','a2_z','a1_y','a1_z'], axis = 1)
df_bearing_1_y = df_train.drop(['a2_x','a2_y','a2_z','a1_x','a1_z'], axis = 1)
df_bearing_1_z = df_train.drop(['a2_x','a2_y','a2_z','a1_x','a1_y'], axis = 1)
df_bearing_2_x = df_train.drop(['a1_x','a1_y','a1_z','a2_y','a2_z'], axis = 1)
df_bearing_2_y = df_train.drop(['a1_x','a1_y','a1_z','a2_x','a2_z'], axis = 1)
df_bearing_2_z = df_train.drop(['a1_x','a1_y','a1_z','a2_x','a2_y'], axis = 1)
bearing_feature = [df_bearing_1_x,df_bearing_1_y,df_bearing_1_z,df_bearing_2_x,df_bearing_2_y,df_bearing_2_z]
list_column = ['a1_x','a1_y','a1_z','a2_x','a2_y','a2_z']

In [42]:
def by_axis_bearing(bearing_feature,i):
    max_list = []
    for index in range(len(bearing_feature)):
        bearing_idx = bearing_feature[index]
        df_bearing = bearing_idx[bearing_idx['bearing_2_id'] == i]
        fft_values = fftpack.fft(df_bearing)
        max_amplitude = np.argmax(np.abs(fft_values))
        max_list.append(max_amplitude)
    return max_list

In [43]:
#def by_bearing(df):

number_bearing = df_train['bearing_2_id'].max()

max_list = by_axis_bearing(bearing_feature,1)
new_set = pd.DataFrame([max_list],columns=list_column,index=[1])

for i in range(2,number_bearing+1):
    max_list = by_axis_bearing(bearing_feature,i)
    temp_set = pd.DataFrame([max_list],columns=list_column)
    new_set = new_set.append(temp_set, ignore_index=True)

In [48]:
d = origin_set.groupby('bearing_2_id').max()['rpm']

In [56]:
d[d>1513]

bearing_2_id
8      90001.350020
11     18000.054000
14     18000.054000
15      7826.097164
17     60000.600006
19     18000.054000
21      6923.084911
23     60000.600006
24     22500.084375
26      1525.423729
29     36000.216001
36     25714.395919
40      1525.423729
54      1525.423729
56      1525.423729
69      1525.423729
80      1525.423729
81     12000.024000
104     1525.423729
105     1525.423729
107     1525.423729
Name: rpm, dtype: float64

In [ ]:
w = np.fft.fft(data)
freqs = fftpack.fftfreq(len(w))
print(freqs.min(), freqs.max())
# (-0.5, 0.499975)

# Find the peak in the coefficients
idx = np.argmax(np.abs(w))
freq = freqs[idx]
freq_in_hertz = abs(freq * 0.001/3)
print(freq_in_hertz)
# 439.8975

In [ ]:
max_fft_set = new_set.groupby(['bearing_2_id']).max()
max_fft_set = rename_column(max_fft_set,"fft_max_")

In [ ]:
exp_set.columns

In [ ]:
max_set = exp_set.groupby(['bearing_2_id']).max()
max_set = rename_column(max_set,"max_")
min_set = exp_set.groupby(['bearing_2_id']).min()
min_set = rename_column(min_set,"min_")
mean_set = exp_set.groupby(['bearing_2_id']).mean()
mean_set = rename_column(mean_set,"mean_")
std_set = exp_set.groupby(['bearing_2_id']).std()
std_set = rename_column(std_set,"std_")
sum_set = exp_set.groupby(['bearing_2_id']).sum()

In [ ]:
df = pd.concat([min_set,max_set,mean_set,std_set,max_fft_set],axis=1)
df['target']= target_set.iloc[1:,1]

In [ ]:
#df = df.drop('min_timestamp',axis=1)
df

In [ ]:
X = df.drop('target',axis=1)
y = df['target']

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=41)

In [ ]:
param_grid = {
    'n_estimators':[200,150,100,50],
    'criterion':['gini','entropy'],
    'max_depth':[2,4,6,8]
}

In [ ]:
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

Model_rf = RandomForestClassifier(random_state=41)
grid = GridSearchCV(estimator=Model_rf, param_grid=param_grid, n_jobs=-1)
grid.fit(X_train, y_train)
Model_rf = grid.best_estimator_
y_pred_train = Model_rf.predict(X_test)
training_data_accuracy_rf = accuracy_score(y_test, y_pred_train)

In [ ]:

print("The accuracy of RandomForestC Model is", (training_data_accuracy_rf*100), '%')
print('The Cross Validation Rapport : ','\n', (classification_report(y_test, y_pred_train)))

In [ ]:
rf = RandomForestClassifier(criterion='entropy', max_depth=6)
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)
print(accuracy_score(y_pred, y_test))
y_train_pre = rf.predict(X_train)
print(accuracy_score(y_train_pre, y_train))

In [ ]:
scores = cross_val_score(rf, X_train, y_train, cv=10, scoring = "accuracy")
print("Scores:", scores)
print("Accuracy_score", accuracy_score(y_train_pre, y_train))
print("Mean:", scores.mean())
print("Standard Deviation:", scores.std())
print("Variation:", scores.var())

In [ ]:
importances = pd.DataFrame({'feature':X_train.columns,'importance':np.round(Model_rf.feature_importances_,3)})
importances1 = importances.sort_values('importance',ascending=False).set_index('feature')
importances1.plot(kind='bar',figsize=(15,6))